# Goal:

- Load a DGL-provided dataset.

- Build a GNN model with DGL-provided neural network modules.

- Train and evaluate a GNN model for node classification on either CPU or GPU.

In [1]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

Using backend: pytorch


In [2]:
import dgl.data

dataset = dgl.data.CoraGraphDataset()
print(f'Number of categories: {dataset.num_classes}')

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Number of categories: 7


In [3]:
g = dataset[0] # dataset has only one graph

A DGL graph can store node features and edge features in two dictionary-like attributes called **ndata** and **edata**. In the DGL Cora dataset, the graph contains the following node features:

- **train_mask**: A boolean tensor indicating whether the node is in the training set.

- **val_mask**: A boolean tensor indicating whether the node is in the validation set.

- **test_mask**: A boolean tensor indicating whether the node is in the test set.

- **label**: The ground truth node category.

- **feat**: The node features.

In [4]:
print('Node features:')
print(g.ndata)
print('Edge features:')
print(g.edata)

Node features:
{'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'train_mask': tensor([ True,  True,  True,  ..., False, False, False])}
Edge features:
{}


# Simple 2 layer Graph Convolutional Network (GCN)

In [5]:
from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(input_dim, hidden_dim)
        self.conv2 = GraphConv(hidden_dim, hidden_dim)
    
    def forward(self, g, input_feat):
        h = self.conv1(g, input_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

In [6]:
def train(g, model, epochs: int):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0
    
    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    
    for epoch in range(epochs):
        # Forward
        logits = model(g, features)
        
        # compute prediction
        pred = logits.argmax(1)
        
        # compute loss (only of nodes in the training set!)
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])
        
        # compute accuracy on train/val/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()
        
        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc
            
        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if epoch % 5 == 0:
            print(f'Epoch: {epoch}, loss: {loss:.3f}, val acc: {val_acc:.3f} (best {best_val_acc:.3f}), test acc: {test_acc:.3f} (best {best_test_acc:.3f})')


In [7]:
g = g.to('cuda')
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes).to('cuda')
train(g, model, 100)

Epoch: 0, loss: 2.773, val acc: 0.136 (best 0.136), test acc: 0.118 (best 0.118)
Epoch: 5, loss: 2.640, val acc: 0.490 (best 0.490), test acc: 0.480 (best 0.480)
Epoch: 10, loss: 2.449, val acc: 0.500 (best 0.506), test acc: 0.486 (best 0.488)
Epoch: 15, loss: 2.225, val acc: 0.436 (best 0.506), test acc: 0.428 (best 0.488)
Epoch: 20, loss: 2.021, val acc: 0.354 (best 0.506), test acc: 0.362 (best 0.488)
Epoch: 25, loss: 1.856, val acc: 0.344 (best 0.506), test acc: 0.355 (best 0.488)
Epoch: 30, loss: 1.718, val acc: 0.480 (best 0.506), test acc: 0.505 (best 0.488)
Epoch: 35, loss: 1.586, val acc: 0.566 (best 0.566), test acc: 0.554 (best 0.554)
Epoch: 40, loss: 1.444, val acc: 0.704 (best 0.704), test acc: 0.704 (best 0.704)
Epoch: 45, loss: 1.291, val acc: 0.746 (best 0.748), test acc: 0.756 (best 0.754)
Epoch: 50, loss: 1.130, val acc: 0.752 (best 0.752), test acc: 0.753 (best 0.757)
Epoch: 55, loss: 0.970, val acc: 0.762 (best 0.762), test acc: 0.761 (best 0.761)
Epoch: 60, loss: 0